## ВКР: обработка данных

Автоматизация подбора персонала

Копчев Владислав, БПМИ197

### Пост-обработка

In [100]:
df = pd.read_csv('resumes_features.csv')

**Перевод опыта работы в числовое значение:**

In [101]:
df['Опыт работы, лет'][2]
opyt_normalized = []
for x in df['Опыт работы, лет']:
    x=x.removeprefix('Work experience ').removeprefix('Опыт работы ')
    x=x.replace('years', ';')
    x=x.replace('year', ';')
    x=x.replace('года', ';')  # ВСЕГДА д. б. раньше
    x=x.replace('год', ';')
    x=x.replace('лет', ';')
    
    if x.find(';') == -1:
        x = '0;' + x
    x=x.removesuffix('months')
    x=x.removesuffix('month')
    x=x.removesuffix('месяцев')
    x=x.removesuffix('месяц')
    x=x.removesuffix('месяца')
     
    x=x.replace(' ', '')
    if re.search('[а–Я]+|[a–Z]+', x):
        x='---'
    opyt_normalized.append(x)
    
df['Опыт нормализованный'] = opyt_normalized
df.drop(list(df)[0], axis=1)

KeyError: 'Опыт работы, лет'

**Словарь образования:**

In [102]:
edu_list = pd.concat([df['Образование-1'], df['Образование-2']], axis=0).drop_duplicates().reset_index().drop('index', axis=1)
edu_list = list(edu_list[list(edu_list)[0]])
len(edu_list)

ys=[]

for x in edu_list:
    s=str(x)
    l = s.find(',')+1
    y=s[l:].lstrip(' ')
    print(end='')
    ys.append(y)
    
for i in range(len(ys)):
    y = ys[i]
    is_space = re.fullmatch('[А-я]+[ ][А-я ()-]+', y)
    if is_space:
        #print(y)
        1==1
    is_m = re.fullmatch('Mosc.*', y) or re.fullmatch('Моск.*', y) or re.fullmatch('Петерб.*', y) or re.fullmatch('Peters.*', y) or re.fullmatch('СПб.*', y) or re.fullmatch('Новосиб.*', y) or re.fullmatch('Novosib.*', y) or re.fullmatch('Томск.*', y) or re.fullmatch('Tomsk.*', y)
    if not is_space and not is_m:
        ys[i] = 'NO'
#ys

cnt=0
for y in ys:
    if y != 'NO':
        cnt+=1
cnt

import numpy as np
ys = np.array(ys)
ys=np.unique(ys)
ys=list(ys)
len(ys)

"""
Топ-10 IT-вузов России по версии QS (Quacquarelli Symonds)

1. Московский государственный университет им. М. В. Ломоносова;
2. Университет ИТМО;
3. Московский физико-технологический институт (МФТИ);
4. Высшая школа экономики;
5. Санкт-Петербургский политехнический университет Петра Великого; 
6. Московский государственный технический университета (МГТУ) имени Н.Э. Баумана;
7. Санкт-Петербургский политехнический университет Петра Великого; 
8. Новосибирский государственный университет;
9. Московский инженерно-физический институт (МИФИ);
10. Томский политехнический университет.

https://propostuplenie.ru/article/spisok-luchshih-informacionnyh-vuzov-rossii/
"""

def is_top(y):
    if re.search('ВШЭ', y) or re.search('Высшая школа экономики', y) or re.search('HSE', y):
        y='ВШЭ'
    elif re.search('Московский Государственный Университет$', y) or re.search('Московский Государственный Университет им[.  ени] (М.|Л)', y) or re.search('МГУ[^ А-я]|\(МГУ\)', y) or re.search('Moscow state University', y) or re.search('MSU', y):
        y='МГУ'
    elif re.search('ИТМО', y):
        y='ИТМО'
    elif re.search('МФТИ', y) or re.search('Московский физико-технологический институт', y):
        y='Физтех'
        # ЗФТШ при МФТИ
        # - Физтех
    elif re.search('СПБ Политех', y) or re.search('Санкт-Петербургский политехнический университет', y):
        y='Политех СПб'
    elif re.search('Бауман', y):
        y='Бауманка'
    elif re.search('НГУ', y) or re.search('Новосибирский государственный университет', y):
        y='НГУ'
        # НГУ Наталии Нестеровой
    elif re.search('МИФИ', y) or re.search('Московский инженерно-физический институт', y):
        y='МИФИ'
        # Какие-то Техникум ДИТИ НИЯУ МИФИ и прочее...
    elif re.search('ТомГУ', y) or re.search('Томский политехнический университет', y):
        y='ТомГУ'
    else:
        y='NO'
    return y != 'NO'
        
# Дальше завожу колонку: parsed_uni по этому правилу -- 1 если хотя бы 1 уник топовый
# И там заменяю значения и вывожу равно ли МГУ / етц.

xs = []
for x in df.index:
    stroke = df.iloc[x].loc[['Образование-1', 'Образование-2']]
    k2 = False
    if type(stroke[0]) != float:
        k1 = is_top(stroke[0])
    if type(stroke[1]) != float:
        k2 = is_top(stroke[1])
    xs.append(k1 or k2)
    
df['Топовость образования']=xs

Необходимо еще доделать правила:

In [103]:
is_top('МГУ') # почему-то не работает
# for x in 

False

**Флаг, что текст на eng:**

In [104]:
df['О себе']
cnt=0

for y in df['О себе']:
    if type(y)!=float:  # not nan
        if re.search('[a–Z]+', y):  # более точно: у английских резюме интересы тоже будут на англ итд, потом подумаю над этим
            cnt+=1
cnt, df.shape[0]-cnt

ts=[]
for x in df['О себе']:
    t = 'NOT ENG, NOT RU'
    if type(x)!=float:  # not nan
        t = re.search('[a–Z]+', x)
        if t:
            t = 'ENG'
        else:
            t = 'RU'
    ts.append(t)
df['is_eng?'] = ts

**Справочник по интересам:**

In [105]:
inter_list = []

for inter in df['Интересы']:
    intr = inter.split('\n')
    if re.search('(Analyst)|(Аналитик данных)|(аналитик)|(Аналитик)', inter):
        inter_list.append('found smt by rule (Analyst)|(Аналитик данных)|(аналитик)|(Аналитик)')
    else:
        inter_list.append('---')
        
df['Справочник по интересам']= inter_list

**Извлечение сущностей:**

In [106]:
xs1 = []
xs2=[]
xs3=[]
for x in df['Коммандировка']:
    # english
    x1 = re.search('[A-z]+', x)
    x2 = re.search('[a-z ]*willing to relocate', x)
    x3 = re.search('[a-z ]*prepared for [a-z]* business trips', x)
    if x1:
        x1=x1[0]
    if x2:
        x2=x2[0]
    if x3:
        x3=x3[0]
    
    if x1 or x2 or x3:  # if found smt in engish — is it not wrong way to do it?
        xs1.append(x1)
        xs2.append(x2)
        xs3.append(x3)
    else:
        # russian — work in progress
        # Москва, не готова к переезду, готова к редким командировкам
        # Санкт-Петербург, м. Гражданский проспект, готов к переезду (Москва), готов к командировкам
        # Москва, готов к переезду (Нидерланды, Испания, Франция, Италия, Австрия, Германия, Швейцария), готов к командировкам
        # Москва, хочу переехать, готов к командировкам
        # Москва, м. Ясенево, не готов к переезду, готов к редким командировкам
        x1 = re.search('[А-я]+', x)  # is it correct?
        x2 = re.search('[А-я]*[ не]*готов[а]* к переезду', x)
        x3 = re.search('[А-я]*[ не]*готов[а]* к[ а-я]* командировкам', x)  # черновое правило
        if x1:
            x1=x1[0]
        if x2:
            x2=x2[0]
        if x3:
            x3=x3[0]
        xs1.append(x1)
        xs2.append(x2)
        xs3.append(x3)

zs = []
for z in df['Навыки']:
    zzz = z.split('\n')
    zs.append(zzz)
    
df['Навыки списком'] = zs
df['Релокация?'] = xs3
df['Коммандировка?'] = xs2
df['Город?'] = xs1

"""
1. Коммандировка 
2. Навыки
3. Кол-во работ
4. Наличие аналитических комбинаций слов в работе — новый столбец
"""

import numpy as np
df['Коммандировка'][8]

# (не) готов -> (не) готова
# (not) willing, (not) preprared

# City, (not willing)., (not) prepared

import re

print(df['Коммандировка'][8])
print(re.search('[A-z]+', df['Коммандировка'][8]))
print(re.search('[a-z ]*willing to relocate', df['Коммандировка'][8]))
print(re.search('[a-z ]*prepared for business trips', df['Коммандировка'][8]))

print(df['Коммандировка'][0])
print(re.search('[A-z]+', df['Коммандировка'][0]))
print(re.search('[a-z ]*willing to relocate', df['Коммандировка'][0]))
print(re.search('[a-z ]*prepared for [a-z]* business trips', df['Коммандировка'][0]))
# ^ оказывается есть еще occasional
# тогда для eng-коммандировки готово!

# Коммандировка более-менее есть. Теперь навыки. 

df['Навыки'][0].split('\n')  # вот так?

# Навыки есть. Теперь кол-во работ. Для этого соскрейпить все работы и их кол-во

df

# У меня уже скрейпится список всех работ. Надо его добавить просто а не только 12!!

# Все вроде я это собрал

# Теперь наличие аналитических комбинаций слов в работе. Для этого сначала сделать 3 пункт!!

# Кол-во работ сделано. Итак:

work_list = df['Где работал?']
analytics_words = []
for work in work_list:
    # пока супер примитивное правило:
    if re.search('(SQL)|(Excel)|(Python)|(Tableau)|(Power BI)', work):
        analytics_words.append('found smt by rule (SQL)|(Excel)|(Python)|(Tableau)|(Power BI)')
    else:
        analytics_words.append('---')
df['Аналитические комбинации слов'] = analytics_words

Moscow, not willing to relocate, prepared for business trips
<re.Match object; span=(0, 6), match='Moscow'>
<re.Match object; span=(7, 31), match=' not willing to relocate'>
<re.Match object; span=(32, 60), match=' prepared for business trips'>
Moscow, willing to relocate, prepared for occasional business trips
<re.Match object; span=(0, 6), match='Moscow'>
<re.Match object; span=(7, 27), match=' willing to relocate'>
<re.Match object; span=(28, 67), match=' prepared for occasional business trips'>


KeyError: 'Где работал?'

**Перевод уровня образования:**

In [107]:
z = list(np.unique(list(df['Уровень образование'])))

for y in z:
    y=y.removeprefix('Higher education ')
    y=y.replace('(', '').replace(')', '')
    y=y.replace('bachelor', 'Бакалавр').replace('master', 'Магистр')
    y=y.replace('Doctor of Science', 'PhD').replace('PhD', 'Доктор наук')
    
    y=y.removeprefix('Высшее образование ')
    y=y.replace('Higher education', 'Высшее образование')
    y=y.replace('Incomplete higher education', 'Неоконченное высшее образование')
    y=y.replace('Secondary education', 'Среднее образование')
    y=y.replace('Secondary special education', 'Среднее специальное образование')
    print(y)
    
ys=[]
for y in df['Уровень образование']:
    y=y.removeprefix('Higher education ')
    y=y.replace('(', '').replace(')', '')
    y=y.replace('bachelor', 'Бакалавр').replace('master', 'Магистр')
    y=y.replace('Doctor of Science', 'PhD').replace('PhD', 'Доктор наук')
    
    y=y.removeprefix('Высшее образование ')
    y=y.replace('Higher education', 'Высшее образование')
    y=y.replace('Incomplete higher education', 'Неоконченное высшее образование')
    y=y.replace('Secondary education', 'Среднее образование')
    y=y.replace('Secondary special education', 'Среднее специальное образование')
    ys.append(y)
df['Уровень оборазования нормализованный'] = ys

Высшее образование
Доктор наук
Доктор наук
Бакалавр
Магистр
Неоконченное высшее образование
Среднее образование
Среднее специальное образование
nan
Высшее образование
Бакалавр
Доктор наук
Кандидат наук
Магистр
Неоконченное высшее образование
Образование
Среднее образование
Среднее специальное образование


AttributeError: 'float' object has no attribute 'removeprefix'

**Сохранение результатов:**

In [92]:
list(df)[:6]

['Unnamed: 0',
 'Ссылка',
 'Название',
 'Коммандировка',
 'Опыт работы, лет',
 'О себе']

In [93]:
# df=df.drop(list(df)[0], axis=1)

In [108]:
df.to_csv('resumes_features.csv')

### Что дальше? (Раньше было частью черновика)

Осталось доделать справочник по образованию еще лучше, а также справочник по интересам и наличие аналитчиеских комбинаций слов (см черновик, что сделать)

In [95]:
# concatennate tables

# df1 = read_csv(resumes_features)

# df2 = gather_data(driver)

# df3 = pd.concat([df1, df2]).drop_duplicates

# !mv resumes_features/backup/cur_date/resumes_features

# df3.to_csv(resumes_features)

**начинаем АД:**

норм RU/ENG:

In [143]:
# old : df = pd.read_csv('resumes_features.csv')
df = pd.read_csv('backup/16.11/resumes_features.csv')
#df = pd.read_csv('resumes_features.csv')

In [144]:
ts = []

for i in range(df.shape[0]):
    if type(df['Город?'][i])!=float and type(df['Релокация?'][i])!=float and type(df['Коммандировка?'][i])!=float:  # not nan\
        t = re.search('[A-z]+', df['Город?'][i]) or re.search('[A-z]+', df['Релокация?'][i]) or re.search('[A-z]+', df['Коммандировка?'][i])
        if t:
            t = 'ENG'
        else:
            t = 'RU'

    # print(x, t)
    else:
        t = '---'
    ts.append(t)
        
df['is_eng?'] = ts

In [145]:
about = df['О себе'].loc[df['is_eng?'] == 'RU']

In [146]:
df = df.loc[df['is_eng?'] == 'RU']

In [147]:
about

1        В последние годы проходила обучение без возмож...
2                                                      ---
3        Имею экономическое образование. Продвинутый по...
4        Имеется опыт в создании Android-приложений на ...
5        Уверенный пользователь.\nЖелание оптимизации и...
                               ...                        
19813    Высокая работоспособность, вдумчивость, уравно...
19814    Увлекаюсь спортом, вредные привычки отсутствую...
19815    Уверенный пользователь ПК, работа с оргтехнико...
19816    Я абсолютно честен и очень ответственен. Люблю...
19817    I. ОБЩАЯ ПОДГОТОВЛЕННОСТЬ И ОПЫТ РАБОТЫ\n\nДел...
Name: О себе, Length: 18236, dtype: object

In [148]:
df

,Unnamed: 0,Unnamed: 0.1,Ссылка,Название,Коммандировка,Опыт,О себе,Образование,Интересы,Навыки,...,Кол-во образований,Работа 1,Работа 2,Топовость образования,is_eng?,Справочник по интересам,Навыки списком,Релокация?,Коммандировка?,Город?
1,1,1,https://hh.ru/resume/628596ac000657935b0039ed1...,BI аналитик,"Москва, не готова к переезду, готова к редким ...",Опыт работы 1 год 5 месяцев,В последние годы проходила обучение без возмож...,Высшее образование (Бакалавр)\n2022\nНациональ...,"Специализации:\nBI-аналитик, аналитик данных\n...","Работа с компьютерными программами, программы ...",...,1,"Ozon\nМладший аналитик\nСоздание, поддержка и ...",OZON\nСтажер группы BI аналитики и отчетности\...,True,RU,found smt by rule (Analyst)|(Аналитик данных)|...,"['Работа с компьютерными программами, программ...",готова к редким командировкам,не готова к переезду,Москва
2,2,2,https://hh.ru/resume/dfbcae000004342d8f0039ed1...,BI аналитик,"Москва, не готова к переезду, готова к команди...",Опыт работы 3 года 3 месяца,---,Высшее образование (Бакалавр)\n2019\nСамарский...,"Специализации:\nАналитик\nBI-аналитик, аналити...",Другое\nPython\nSQL\nУмение работать в коллект...,...,1,"Банк ВТБ (ПАО)\nМосква, www.vtbcareer.com\nФин...",---,False,RU,found smt by rule (Analyst)|(Аналитик данных)|...,"['Другое', 'Python', 'SQL', 'Умение работать в...",готова к командировкам,не готова к переезду,Москва
3,3,3,https://hh.ru/resume/bebf6a630008069d8c0039ed1...,Аналитик,"Санкт-Петербург, м. Гражданский проспект, гото...",Опыт работы 7 лет 2 месяца,Имею экономическое образование. Продвинутый по...,Высшее образование (Бакалавр)\n2015\nСанкт-Пет...,Специализации:\nАналитик\nЗанятость: полная за...,"Работа с компьютерными программами, программы ...",...,1,A&D Mortgage\nСтарший BI аналитик\nСоздание Po...,"Сбер\nСанкт-Петербург, rabota.sber.ru/\nФинанс...",False,RU,found smt by rule (Analyst)|(Аналитик данных)|...,"['Работа с компьютерными программами, программ...",готов к командировкам,готов к переезду,Санкт
4,4,4,https://hh.ru/resume/8606f37b000773de980039ed1...,Программист-разработчик,"Москва, м. Петровско-Разумовская, не готов к п...",Опыт работы 1 год 1 месяц,Имеется опыт в создании Android-приложений на ...,Высшее образование (Бакалавр)\n2022\nЧелябинск...,"Специализации:\nПрограммист, разработчик\nЗаня...",Применение стека технологий/Цифровые навыки\nL...,...,1,MoneyCare\nBI-аналитик\nРабота с big data(Pyth...,---,False,RU,---,['Применение стека технологий/Цифровые навыки'...,не готов к командировкам,не готов к переезду,Москва
5,5,5,https://hh.ru/resume/993bd0980002dd6cff0039ed1...,Аналитик Bi,"Москва, м. Сокол, готов к переезду, готов к ко...",Опыт работы 10 лет 7 месяцев,Уверенный пользователь.\nЖелание оптимизации и...,Высшее образование\n2010\nДальневосточный госу...,"Специализации:\nBI-аналитик, аналитик данных\n...","Работа с компьютерными программами, программы ...",...,1,Brixo\nАналитик отдела продаж\nФормирование и ...,"Смарт индастриз\nМеталлургия, металлообработка...",False,RU,found smt by rule (Analyst)|(Аналитик данных)|...,"['Работа с компьютерными программами, программ...",готов к командировкам,готов к переезду,Москва
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19813,19813,19813,https://hh.ru/resume/57e8fd3c000211f20a0039ed1...,Финансовый директор,"Москва, м. Тушинская, не готов к переезду, гот...",Опыт работы 25 лет 3 месяца,"Высокая работоспособность, вдумчивость, уравно...",Высшее образование\n2013\nЕдиная Аттестационна...,Специализации:\nФинансовый менеджер\nЗанятость...,---,...,2,"ЗАО ""Аудит БОС""\nМосква, audit-bos.ru\nФинансо...",---,False,RU,---,['---'],готов к редким командировкам,не готов к переезду,Москва
19814,19814,19814,https://hh.ru/resume/bd1a048a0002633bc20039ed1...,Аналитик,"Оренбург, готов к переезду (Москва), не готов ...",Опыт работы 10 лет 8 месяцев,"Увлекаюсь спортом, вредные привычки отсутствую...","Высшее образование\n2011\nОГАУ\nЭкономический,...",Специализации:\nАналитик\nЗа

ключевые совпадения см л34 она скинула

**Извлечение сущностей: более соершенные правила.** ВКР. Если город и коммандировка на английском то резюме точно на английском. Какие-то такие правила еще поискать. Посмотрю только в каком таком точном поле меньше всего пропусков. ВКР: позже вычленяю ещё сущности как мы обсуждали см prev call в 7.12. См github: допилить еще справочник образования и прочее, чтобы все в этом плане было идеально

В `scraping.ipynb` были собраны и подготовлены данные. Возможно, стоит еще доработать (в разделе "пост-обработка"):
- Словарь образования
- Справочник по интересам
- Аналитические комбинации слов

**теперь немного про данные:**

```
#df['Работа']
list(df)
```

для списка карьерного роста надо будет собрать список компаний где был и найти там совпадения, на текущих данных так сделать нельзя

In [40]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
# !pip install transliterate
from transliterate import translit
import re
import numpy as np

In [41]:
import nltk
# nltk.download('stopwords')
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
morph = MorphAnalyzer()

In [48]:
driver = webdriver.Chrome('./chromedriver')

/var/folders/0k/pk7zyw6x2sqcjpbqvgf0ssh80000gn/T/ipykernel_27753/2299280398.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [49]:
repet = []

In [50]:
for i in range(df.shape[0]):
    driver.get(df.iloc[i]['Ссылка'])
    work = driver.find_elements(by=By.XPATH, 
                             value='//div[@data-qa="resume-block-experience"][@class="resume-block"]//div[@class="resume-block-container"]')
    w = driver.find_elements(by=By.XPATH, 
                             value='//div[@data-qa="resume-block-experience"][@class="resume-block"]//div[@class="resume-block-container"]/div[@class="bloko-text bloko-text_strong"][not(@data-qa="resume-block-experience-position")]//span')
    
    works = []
    for ww in w:
        works.append(ww.text)
    
    works_normilized = []
    for w in works:
        works_normilized.append(morph.normal_forms(w)[0])
    
    repet.append(len(set(works_normilized)) - len(works_normilized) < 0)

KeyboardInterrupt: 

проверить, правда ли так много людей с повышениями:

In [51]:
repet

[True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False]

теперь для нормализованных названий найти повторения: `len(set(works)) - len(works) > 0` — и чекнуть адекватность такого определения

^ почти готово

_куда-то надо сохранить этот новый датасет в приличное место (или вообще туда же, просто старую версию датасета переместить в backup)_:

In [ ]:
# dataset.save()

а также я давно хотел заняться топовостью обоазования

вообще там вроде все ок, тогда посмотрю вручную при разметке есть ли проблемы и почему, составлю список прямо

**Какой был план:**
    
    ВКР: размечу данные x100 -> соберу ещё признаки нормально x2 -> обучу регрессию и получу результат + проинтерпретирую кластеры -> дальше бустинг -> semi-supervised, сложные методы кластеризации. Что еще можно?

#### Предобработка полученных данных для анализа

Токенизация раздела "О себе" на 1-граммы:

In [12]:
import nltk
# nltk.download('stopwords')

import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
patterns = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()
def lemmatize(doc):
    # print(doc)
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru and morph.normal_forms(token)[0] not in stopwords_ru:  # ВОТ откуда там стоп-слвоа — В not in stopwords_ru -> normal(В) = в
            # print(token)
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

df2 = pd.DataFrame(df['О себе'])

for col_ind in df2.index:
    col = df2.iloc[col_ind]['О себе']
    if col:
        df2.iloc[col_ind]['О себе'] = lemmatize(str(col))
        #print(lemmatize(str(col)))
        
data = df2['О себе']

from collections import defaultdict
word_freq = defaultdict(int)
for tokens in data.iloc[:]:
    if tokens:
        for token in tokens:
            word_freq[token] += 1

In [13]:
df2.sample(3)

,О себе
166,"[усидчивый, целеустремлённый, адекватный, поль..."
19257,"[владеть, навык, финансовый, анализ, банк, ком..."
9120,"[management, and, business, skills, project, m..."


^ Стоп-слова убраны, все в нормальной форме => их не будет и дальше.

На 2-граммы:

In [14]:
from nltk.util import ngrams
from nltk import word_tokenize 

tokens2 = []

for x in df2['О себе']:
    if x is not None:
        bigrams = []
        for bg in list(ngrams(x, 2)):
            bigrams.append(' '.join(bg))
        tokens2.append(bigrams)
    else:
        tokens2.append(None)
        
df2['О себе 2'] = tokens2

In [15]:
df2.sample(3)

,О себе,О себе 2
13151,"[•, готовность, активный, работа, •, стремлени...","[• готовность, готовность активный, активный р..."
15309,"[отсутствие, вредный, привычка, энергичность, ...","[отсутствие вредный, вредный привычка, привычк..."
11546,"[уверенный, пользователь, пк, word, excel, out...","[уверенный пользователь, пользователь пк, пк w..."


На 3-граммы:

In [16]:
from nltk.util import ngrams
from nltk import word_tokenize 

tokens3 = []

for x in df2['О себе']:
    if x is not None:
        trigrams = []
        for tg in list(ngrams(x, 3)):
            trigrams.append(' '.join(tg))
        tokens3.append(trigrams)
    else:
        tokens3.append(None)
        
df2['О себе 3'] = tokens3

In [17]:
df2.sample(3)

,О себе,О себе 2,О себе 3
11691,"[любить, музыка, достигать, поставить, результ...","[любить музыка, музыка достигать, достигать по...","[любить музыка достигать, музыка достигать пос..."
11665,"[хобби, спорт, доброжелательный, чувство, юмор...","[хобби спорт, спорт доброжелательный, доброжел...","[хобби спорт доброжелательный, спорт доброжела..."
13230,None,None,None


Строим эмбеддинги:

In [18]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

Избавились от пропусков:

In [19]:
df2.loc[df2['О себе 3'].isna(), 'О себе 3'] = '-'

In [20]:
df2[df2['О себе 3'].isna()]['О себе 3']

Series([], Name: О себе 3, dtype: object)

In [21]:
model = Word2Vec(sentences=df2['О себе 3'], vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [22]:
model = Word2Vec.load("word2vec.model")

In [23]:
model

In [24]:
model.train(df2['О себе 3'], total_examples=len(df2['О себе 3']), epochs=1)

(956003, 956134)

^ наверное лучше еще привести все токены в тексте к норм. форме?

In [25]:
df2['О себе 3']

0        [responsible communicable quick, communicable ...
1        [последний год проходить, год проходить обучен...
2                                                        -
3        [иметь экономический образование, экономически...
4        [иметься опыт создание, опыт создание android,...
                               ...                        
19813    [высокий работоспособность вдумчивость, работо...
19814    [увлекаться спорт вредный, спорт вредный привы...
19815    [уверенный пользователь пк, пользователь пк ра...
19816    [абсолютно честный очень, честный очень ответс...
19817    [i общий подготовленность, общий подготовленно...
Name: О себе 3, Length: 19818, dtype: object

In [26]:
vector = model.wv['последний год проходить']  # get numpy vector of a word
sims = model.wv.most_similar('последний год проходить', topn=10)  # get other similar words

In [27]:
sims

[('обучение возможность работать', 0.9961733818054199),
 ('проходить обучение возможность', 0.9960487484931946),
 ('год проходить обучение', 0.9890427589416504),
 ('возможность работать проходить', 0.9108878374099731),
 ('возможность работать заниматься', 0.8885632753372192),
 ('проходить обучение университет', 0.8795680999755859),
 ('год проходить служба', 0.8522109389305115),
 ('возможность работать специальность', 0.8485803008079529),
 ('возможность работать последний', 0.8074821829795837),
 ('привычка последний год', 0.7890995144844055)]

In [28]:
vector

array([-3.60620886e-01, -1.19415365e-01, -2.84469426e-01, -2.48883486e-01,
       -2.51464814e-01, -6.08071610e-02, -1.04758432e-02, -6.53371438e-02,
        2.79351324e-01,  1.10671245e-01, -5.84037781e-01,  1.16823658e-01,
        3.39617103e-01, -2.68100441e-01, -7.64075592e-02, -3.73274088e-01,
       -1.50579855e-01, -7.40459114e-02, -2.17869386e-01,  1.18627539e-02,
        2.96194434e-01,  2.87683606e-01,  8.75545591e-02, -1.33229315e-01,
        4.55688417e-01, -1.57081485e-01,  6.04974568e-01,  1.45652384e-01,
       -3.84899855e-01, -1.49883494e-01, -7.66195580e-02,  2.68475786e-02,
        1.03311688e-01, -3.54715645e-01,  4.29094881e-02,  8.73202682e-02,
       -2.88378000e-01,  1.41304672e-01,  2.00332776e-02,  9.77834966e-03,
       -1.14594050e-01,  3.36564153e-01, -1.43547893e-01, -5.12990296e-01,
       -1.30183131e-01, -1.10521860e-01,  4.48991805e-02, -8.41734558e-02,
        9.48970169e-02, -2.32677042e-01,  6.44028544e-01, -3.39331627e-01,
       -2.12573841e-01,  

Итак, мы построили эмбеддинги. Теперь кластеризуем. Для этого сначала строим новую табличку:

In [29]:
model.wv

In [30]:
our_data = pd.DataFrame(df2['О себе 3'])

In [31]:
our_data['emb'] = 0

In [32]:
our_data

,О себе 3,emb
0,"[responsible communicable quick, communicable ...",0
1,"[последний год проходить, год проходить обучен...",0
2,-,0
3,"[иметь экономический образование, экономически...",0
4,"[иметься опыт создание, опыт создание android,...",0
...,...,...
19813,"[высокий работоспособность вдумчивость, работо...",0
19814,"[увлекаться спорт вредный, спорт вредный привы...",0
19815,"[уверенный пользователь пк, пользователь пк ра...",0
19816,"[абсолютно честный очень, честный очень ответс...",0


In [33]:
embs = []
for x in our_data['О себе 3']:
    cur_embs = []
    for y in x:
        cur_embs.append(model.wv[y])
    embs.append(cur_embs)

In [34]:
our_data['emb'] = embs

теперь надо построить `flat_emb` — по векторам эмбеддингов построить единый вектор эмбеддингов.

2ая попытка:

In [35]:
# буду использовать среднее

embs_flattened = []
for x in our_data['emb']:
    x_flat = np.array(x[0])
    for z in x[1:]:
        x_flat += np.array(z)
    embs_flattened.append(x_flat / len(x))

In [36]:
embs_flattened[4].size

100

почему это работает: Для ВКР и IR одновременно: https://stats.stackexchange.com/questions/318882/what-does-average-of-word2vec-vector-mean

1ая попытка:

```
embs_flattened = []
for x in our_data['emb']:
    x_flat = []
    for z in x:
        x_flat += list(z)
    embs_flattened.append(x_flat)
```

после поаытки:

In [37]:
our_data['flat_emb'] = embs_flattened

в чем проблема с `flat_emb` как эмбеддингами??? именно поэтому не работает kmeans.

Итак, у меня для каждого токена в тексте есть эмбеддинг. Как из этого получить что-то адекватное?

In [38]:
our_data['flat_emb'][0]

array([-1.0651934e-03, -9.5223321e-04, -2.6917730e-03,  2.1369762e-03,
       -5.8708451e-04, -2.7602646e-03,  5.7263573e-04,  5.7984889e-04,
        3.4672272e-04, -2.4817343e-04, -2.1509365e-03, -1.0595277e-03,
        2.2162371e-03,  4.3668365e-03, -1.9517433e-03, -1.6647292e-03,
        9.2457375e-04, -4.3015121e-04,  7.9902558e-04, -1.8386915e-03,
       -1.9029748e-03, -4.1571790e-03, -1.4351994e-03, -8.0903992e-04,
       -4.2714077e-04,  1.2432009e-03,  2.4210550e-03,  1.8813036e-03,
        6.1120614e-03,  1.8388812e-03, -4.6735100e-04, -2.8745071e-03,
       -1.2642957e-03, -5.9631222e-04, -1.0154785e-03,  2.3797858e-03,
       -1.9074589e-03,  1.8787604e-03, -1.1865874e-03, -1.1884656e-04,
        1.5889053e-04,  2.5906737e-03, -5.7543819e-03,  2.9124215e-04,
        3.0745247e-03, -1.2872546e-03, -2.3613067e-03, -2.7564031e-03,
       -5.0969948e-03,  9.5343683e-05,  2.4627807e-04,  5.7958264e-04,
        7.2839612e-04, -4.5149151e-04,  1.7596272e-04, -1.5821084e-04,
      

In [39]:
our_data['flat_emb'].dropna()

0        [-0.0010651934, -0.0009522332, -0.002691773, 0...
1        [-0.037309267, -0.009970679, -0.027855689, -0....
2        [-0.00053622725, 0.00023643016, 0.0051033497, ...
3        [0.0060664425, 0.0068842797, -0.002567386, 0.0...
4        [-0.0017125845, -0.0012229249, 0.0008647318, -...
                               ...                        
19813    [0.000529979, -0.0016210346, -7.6403834e-05, -...
19814    [-0.00094596954, -0.004241648, -0.001166377, -...
19815    [0.00011770878, 0.033511918, 0.009846648, -0.0...
19816    [-0.0011689814, -0.00021927932, -0.0011205778,...
19817    [-0.0006386235, -0.0001438265, 0.00010306096, ...
Name: flat_emb, Length: 19818, dtype: object

In [40]:
our_data['flat_emb'] * 2

0        [-0.002130387, -0.0019044664, -0.005383546, 0....
1        [-0.07461853, -0.019941358, -0.055711377, -0.0...
2        [-0.0010724545, 0.00047286032, 0.010206699, 0....
3        [0.012132885, 0.013768559, -0.005134772, 0.005...
4        [-0.003425169, -0.0024458498, 0.0017294636, -0...
                               ...                        
19813    [0.001059958, -0.0032420692, -0.00015280767, -...
19814    [-0.0018919391, -0.008483296, -0.002332754, -0...
19815    [0.00023541757, 0.067023836, 0.019693296, -0.0...
19816    [-0.0023379629, -0.00043855864, -0.0022411556,...
19817    [-0.001277247, -0.000287653, 0.00020612191, 0....
Name: flat_emb, Length: 19818, dtype: object

In [41]:
our_data['flat_emb']

0        [-0.0010651934, -0.0009522332, -0.002691773, 0...
1        [-0.037309267, -0.009970679, -0.027855689, -0....
2        [-0.00053622725, 0.00023643016, 0.0051033497, ...
3        [0.0060664425, 0.0068842797, -0.002567386, 0.0...
4        [-0.0017125845, -0.0012229249, 0.0008647318, -...
                               ...                        
19813    [0.000529979, -0.0016210346, -7.6403834e-05, -...
19814    [-0.00094596954, -0.004241648, -0.001166377, -...
19815    [0.00011770878, 0.033511918, 0.009846648, -0.0...
19816    [-0.0011689814, -0.00021927932, -0.0011205778,...
19817    [-0.0006386235, -0.0001438265, 0.00010306096, ...
Name: flat_emb, Length: 19818, dtype: object

In [42]:
X = list(our_data['flat_emb'].dropna())

In [43]:
X = np.asarray(X)

In [44]:
X

array([[-0.00106519, -0.00095223, -0.00269177, ...,  0.00334533,
         0.00450045, -0.00186767],
       [-0.03730927, -0.00997068, -0.02785569, ..., -0.01067496,
        -0.03065858, -0.01643043],
       [-0.00053623,  0.00023643,  0.00510335, ..., -0.00704156,
         0.00090146,  0.00639253],
       ...,
       [ 0.00011771,  0.03351192,  0.00984665, ..., -0.04232645,
        -0.01937321, -0.00379713],
       [-0.00116898, -0.00021928, -0.00112058, ...,  0.00060235,
        -0.0007945 , -0.00159538],
       [-0.00063862, -0.00014383,  0.00010306, ..., -0.00021724,
         0.00031917, -0.00088251]], dtype=float32)

In [45]:
for x in X:

IndentationError: expected an indented block (1027233577.py, line 1)

In [46]:
X = np.array([np.array([1, 2, 3]), np.array([2, 3, 4]), np.array([5, 6, 7])])

In [47]:
embs_flattened[0]

array([-1.0651934e-03, -9.5223321e-04, -2.6917730e-03,  2.1369762e-03,
       -5.8708451e-04, -2.7602646e-03,  5.7263573e-04,  5.7984889e-04,
        3.4672272e-04, -2.4817343e-04, -2.1509365e-03, -1.0595277e-03,
        2.2162371e-03,  4.3668365e-03, -1.9517433e-03, -1.6647292e-03,
        9.2457375e-04, -4.3015121e-04,  7.9902558e-04, -1.8386915e-03,
       -1.9029748e-03, -4.1571790e-03, -1.4351994e-03, -8.0903992e-04,
       -4.2714077e-04,  1.2432009e-03,  2.4210550e-03,  1.8813036e-03,
        6.1120614e-03,  1.8388812e-03, -4.6735100e-04, -2.8745071e-03,
       -1.2642957e-03, -5.9631222e-04, -1.0154785e-03,  2.3797858e-03,
       -1.9074589e-03,  1.8787604e-03, -1.1865874e-03, -1.1884656e-04,
        1.5889053e-04,  2.5906737e-03, -5.7543819e-03,  2.9124215e-04,
        3.0745247e-03, -1.2872546e-03, -2.3613067e-03, -2.7564031e-03,
       -5.0969948e-03,  9.5343683e-05,  2.4627807e-04,  5.7958264e-04,
        7.2839612e-04, -4.5149151e-04,  1.7596272e-04, -1.5821084e-04,
      

In [48]:
len(embs_flattened[0])

100

In [49]:
X = embs_flattened

In [50]:
for x in our_data['emb'][0]:
    print(len(x))

100
100
100
100
100


Теперь осталось обработать mBART -> это анаизировать в `analysis.ipynb`

**mBART**

https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta?text=%D0%97%D0%B0+%D0%B2%D1%80%D0%B5%D0%BC%D1%8F+%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%8B+%D0%B8+%D1%83%D1%87%D1%91%D0%B1%D1%8B%2C+%D1%8F+%D0%BE%D0%B2%D0%BB%D0%B0%D0%B4%D0%B5%D0%BB+%D0%BA%D0%BE%D0%BC%D0%BF%D0%B5%D1%82%D0%B5%D0%BD%D1%86%D0%B8%D1%8F%D0%BC%D0%B8%2C+%D0%BD%D0%B5%D0%BE%D0%B1%D1%85%D0%BE%D0%B4%D0%B8%D0%BC%D1%8B%D0%BC%D0%B8+%D0%B4%D0%BB%D1%8F+%D0%B8%D1%81%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F%2C+%D1%81%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F+%D0%B8+%D1%83%D1%81%D0%BF%D0%B5%D1%88%D0%BD%D0%BE%D0%B3%D0%BE+%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F+%D1%86%D0%B8%D1%84%D1%80%D0%BE%D0%B2%D1%8B%D1%85+%D1%82%D0%B5%D1%85%D0%BD%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D0%B9+%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F%2C+%D1%83%D0%BF%D1%80%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F%2C+%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0+%D0%B8+%D1%81%D0%B8%D0%BD%D1%82%D0%B5%D0%B7%D0%B0+%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D1%8B%D1%85+%D1%80%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B9+%D0%B2+%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%B5%D1%81%D0%B0%D1%85+%D1%81%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F+%D1%81%D0%BB%D0%BE%D0%B6%D0%BD%D0%BE%D0%B9+%D0%BF%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%86%D0%B8%D0%B8.%0A%D0%AF+%D0%BE%D0%B1%D0%BB%D0%B0%D0%B4%D0%B0%D1%8E+%D1%88%D0%B8%D1%80%D0%BE%D0%BA%D0%B8%D0%BC+%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%BE%D0%BD%D0%B0%D0%BB%D0%BE%D0%BC+%D0%B2+%D0%BF%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BD%D0%B8%D0%B8+BI+%D0%BE%D1%82%D1%87%D1%91%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%B8%2C+%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B8%2C+%D1%81%D0%B2%D1%8F%D0%B7%D1%8F%D1%85%2C+%D0%BA%D0%BE%D1%80%D1%80%D0%B5%D0%BA%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B8+%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85+%D0%B2+Power+Qwery+%D0%B8+%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%8B+%D1%81+%D0%B1%D0%B0%D0%B7%D0%B0%D0%BC%D0%B8+%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.%0A%D0%A3%D1%87%D0%B0%D1%81%D1%82%D0%B2%D0%BE%D0%B2%D0%B0%D0%BB+%D0%B2+%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%B0%D1%85+%D0%BF%D0%BE+%D0%BE%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8+%D0%B8+%D0%B0%D0%B2%D1%82%D0%BE%D0%BC%D0%B0%D1%82%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8+%D0%B1%D0%B8%D0%B7%D0%BD%D0%B5%D1%81-%D0%BF%D1%80%D0%BE%D1%86%D0%B5%D1%81%D1%81%D0%BE%D0%B2+%D0%B8+%D0%BE%D1%82%D1%87%D1%91%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%B8.%0A%D0%A0%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%BB+%D0%B2+%D0%BA%D0%BE%D0%BC%D0%BF%D0%B0%D0%BD%D0%B8%D1%8F%D1%85+%D1%81+%D1%82%D0%B0%D0%BA%D0%B8%D0%BC%D0%B8+%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B0%D0%BC%D0%B8+%D0%BA%D0%B0%D0%BA+%3A+Power+BI%2C+Excel%2C+MS+Office%2C+VipPlanner.%0A%D0%97%D0%BD%D0%B0%D0%BA%D0%BE%D0%BC+%D1%81+%D1%82%D0%B0%D0%BA%D0%B8%D0%BC%D0%B8+%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%BD%D1%8B%D0%BC%D0%B8+%D0%B8%D0%BD%D1%81%D1%82%D1%80%D1%83%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D0%BC%D0%B8+%D0%BA%D0%B0%D0%BA+%3A+Jira%2C+kanban%2C+confluence.%0A%D0%9F%D1%80%D0%B8%D0%BC%D0%B5%D1%80%D1%8B+%D0%B4%D0%B0%D1%88%D0%B1%D0%BE%D1%80%D0%B4%D0%BE%D0%B2+%D0%BC%D0%BE%D0%B3%D1%83+%D0%B2%D1%8B%D1%81%D0%BB%D0%B0%D1%82%D1%8C+%D0%BB%D0%B8%D1%87%D0%BD%D0%BE.%0A%D0%91%D1%83%D0%B4%D1%83+%D1%80%D0%B0%D0%B4+%D0%B2+%D1%80%D0%B0%D0%BC%D0%BA%D0%B0%D1%85+%D1%81%D0%BE%D0%B1%D0%B5%D1%81%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F+%D0%BE%D1%82%D0%B2%D0%B5%D1%82%D0%B8%D1%82%D1%8C+%D0%BD%D0%B0+%D0%B2%D1%81%D0%B5+%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%B5%D1%81%D1%83%D1%8E%D1%89%D0%B8%D0%B5+%D0%92%D0%B0%D1%81+%D0%B2%D0%BE%D0%BF%D1%80%D0%BE%D1%81%D1%8B.%0A%D0%97%D0%B0%D1%80%D0%B0%D0%BD%D0%B5%D0%B5+%D0%B1%D0%BB%D0%B0%D0%B3%D0%BE%D0%B4%D0%B0%D1%80%D1%8E+%D0%92%D0%B0%D1%81+%D0%B7%D0%B0+%D1%83%D0%B4%D0%B5%D0%BB%D1%91%D0%BD%D0%BD%D0%BE%D0%B5+%D0%BC%D0%BE%D0%B5%D0%B9+%D0%BA%D0%B0%D0%BD%D0%B4%D0%B8%D0%B4%D0%B0%D1%82%D1%83%D1%80%D0%B5+%D0%B2%D1%80%D0%B5%D0%BC%D1%8F.%0A%D0%92%D0%BB%D0%B0%D0%B4%D0%B5%D1%8E+%D0%BD%D0%B0%D0%B2%D1%8B%D0%BA%D0%BE%D0%BC+%22%D0%94%D0%B5%D0%BB%D0%BE%D0%B2%D0%B0%D1%8F+%D0%BF%D0%B5%D1%80%D0%B5%

попробуем:

In [8]:
print(df['О себе'][14])

За время работы и учёбы, я овладел компетенциями, необходимыми для исследования, создания и успешного использования цифровых технологий моделирования, управления, анализа и синтеза системных решений в интересах создания сложной продукции.
Я обладаю широким функционалом в построении BI отчётности, моделировании, связях, корректировании данных в Power Qwery и работы с базами данных.
Участвовал в проектах по оптимизации и автоматизации бизнес-процессов и отчётности.
Работал в компаниях с такими программами как : Power BI, Excel, MS Office, VipPlanner.
Знаком с такими программными инструментами как : Jira, kanban, confluence.
Примеры дашбордов могу выслать лично.
Буду рад в рамках собеседования ответить на все интересующие Вас вопросы.
Заранее благодарю Вас за уделённое моей кандидатуре время.
Владею навыком "Деловая переписка".
Управление проектами PMBOK.

Я люблю читать книги по самосовершенствованию ("год личной эффективности", "Scrum. Революционный метод управления проектами", "Пять ур

In [4]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

In [5]:
model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

^ ок, снизу ... просто тест

In [6]:
article_text = "..."

input_ids = tokenizer(
    [article_text],
    max_length=600,
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True)
print(summary)

una Очередной flash-drive-drive-by-drive-in-the-backs-of-background-of-the-drive-back-by-back-up-for-back-back-of-unside-back-without-backpacking-driven-back-for-driving-back-in-back-to-back-out-of-of-Backpacking-with-back-on-backback-with-with-out-backbackback-ofback-back.


Дальше: article_text = `df['О себе'][0]`; мб надо будет по частям обрабатывать ибо это будет долго и затратно по памяти чтобы ничего не упало и не испортило всю работу